In [ ]:
import os
import fnmatch
import cv2
import numpy as np
import string
import time
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.enable_eager_execution()
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from IPython.display import HTML, display
import time

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))



In [ ]:



#ignore warnings in the output
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:

from tensorflow.python.client import device_lib

# Check all available devices if GPU is available
print(device_lib.list_local_devices())
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9344941547171870114
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6627143861116710704
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 3253136720051650699
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15695549568
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13624947861441220974
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capa

# Data preparation

Required functions

In [ ]:

char_list = list('-0123456789')

 
def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)
        
    return dig_lst

Loading labels

In [ ]:
import pandas as pd
paths = pd.read_csv('/content/drive/My Drive/15k toloka/checked_images.csv')#,keep_default_na=False)
paths = paths.dropna()
# paths.fillna('')

Number of training samples 

In [ ]:
len(paths.index)

6825

We can load data from images or from npz file

In [ ]:
path = '/content/drive/My Drive/15k toloka/images/data/'
 
 
# lists for training dataset
training_img = []
training_txt = []
train_input_length = []
train_label_length = []
orig_txt = []
 
#lists for validation dataset
valid_img = []
valid_txt = []
valid_input_length = []
valid_label_length = []
valid_orig_txt = []
 
max_label_len = 0
read_again = False


if read_again == False:
    files = np.load("./drive/My Drive/15k toloka/dat.npz",allow_pickle=True)

    valid_orig_txt = files["arr_0"]
    valid_label_length = files["arr_1"]
    valid_input_length = files["arr_2"]
    valid_img = files["arr_3"]
    valid_txt = files["arr_4"]
    orig_txt = files["arr_5"]
    train_label_length = files["arr_6"]
    train_input_length = files["arr_7"]
    training_img = files["arr_8"]
    training_txt = files["arr_9"]
    max_label_len = max(valid_label_length.max() , train_label_length.max())

else:
    i =1 
    flag = 0

    n_samples = 6825
    out = display(progress(0, n_samples-1), display_id=True)

        


    for index, row in paths.iterrows():
        out.update(progress(i, n_samples))
        # print(f_name, flush = True)
        # read input image and convert into gray scale image
        f_name = row["file"]
        img = cv2.cvtColor(cv2.imread(os.path.join(path, f_name)), cv2.COLOR_BGR2GRAY)   
        # convert each image of shape (32, 128, 1)
        w, h = img.shape

        
        

        img = cv2.resize(img, dsize=(128, 32), interpolation=cv2.INTER_CUBIC)
        img = np.expand_dims(img , axis = 2)

        # Normalize each image
        img = img/255.
        
        # get the text from the image
        txt = str(int(row["label"]))
        # print(f_name.split('_'), flush = True)
        # compute maximum length of the text
        # print(txt)
        if len(txt) > max_label_len:
            max_label_len = len(txt)
            
            
        # split the 150000 data into validation and training dataset as 10% and 90% respectively
        if i%5 == 0:     
            valid_orig_txt.append(txt)   
            valid_label_length.append(len(txt))
            valid_input_length.append(31)
            valid_img.append(img)
            valid_txt.append(encode_to_labels(txt))
        else:
            orig_txt.append(txt)   
            train_label_length.append(len(txt))
            train_input_length.append(31)
            training_img.append(img)
            training_txt.append(encode_to_labels(txt)) 
        
        # break the loop if total data is 150000
        if i == n_samples:
            flag = 1
            break
        i+=1
        if flag == 1:
            break







In [ ]:
# pad each output label to maximum text length
 
train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))
valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value = len(char_list))

# Model

In [ ]:
# input with shape of height=32 and width=128 
inputs = Input(shape=(32,128,1))
 
# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
batch_norm_1 = BatchNormalization()(pool_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(batch_norm_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
batch_norm_2 = BatchNormalization()(pool_2)

 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(batch_norm_2)
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
# poolig layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
batch_norm_3 = BatchNormalization()(pool_4)


 
conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_3)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)
 
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
 
conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
 
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)
 
# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

# model to be used at test time
act_model = Model(inputs, outputs)

In [ ]:
act_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 128, 1)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 128, 64)       640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 64, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 64, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 64, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 32, 128)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 32, 128)       

In [ ]:
labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
# input_length = Input(name='input_length', shape=[1], dtype='int64')
input_length = Input(name='input_length', shape=[1])
label_length = Input(name='label_length', shape=[1])
# label_length = Input(name='label_length', shape=[1], dtype='int64')
 
 
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
 
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

#model to be used at training time
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

In [ ]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')
 
filepath="best_model.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

In [ ]:
training_img = np.array(training_img)
train_input_length = np.array(train_input_length).astype(float)
train_label_length = np.array(train_label_length).astype(float)

valid_img = np.array(valid_img)
valid_input_length = np.array(valid_input_length)
valid_label_length = np.array(valid_label_length)

# Training

In [ ]:
batch_size = 100
epochs = 500
history = model.fit(x=[training_img, train_padded_txt, train_input_length, train_label_length], y=np.zeros(len(training_img)), batch_size=batch_size, epochs = epochs, validation_data = ([valid_img, valid_padded_txt, valid_input_length, valid_label_length], [np.zeros(len(valid_img))]), verbose = 1, callbacks = callbacks_list)

Train on 5460 samples, validate on 1365 samples
Epoch 1/500
5460/5460 [==============================] - 19s 3ms/step - loss: 8.4254 - val_loss: 14.3094

Epoch 00001: val_loss improved from inf to 14.30942, saving model to best_model.hdf5
Epoch 2/500
5460/5460 [==============================] - 15s 3ms/step - loss: 3.5018 - val_loss: 24.3145

Epoch 00002: val_loss did not improve from 14.30942
Epoch 3/500
5460/5460 [==============================] - 14s 3ms/step - loss: 0.9864 - val_loss: 27.0605

Epoch 00003: val_loss did not improve from 14.30942
Epoch 4/500
5460/5460 [==============================] - 14s 3ms/step - loss: 0.7188 - val_loss: 25.5702

Epoch 00004: val_loss did not improve from 14.30942
Epoch 5/500
5460/5460 [==============================] - 15s 3ms/step - loss: 0.6074 - val_loss: 28.6193

Epoch 00005: val_loss did not improve from 14.30942
Epoch 6/500
5460/5460 [==============================] - 15s 3ms/step - loss: 0.5315 - val_loss: 23.3049

Epoch 00006: val_loss d

In [ ]:
a = model.evaluate(x=[training_img, train_padded_txt, train_input_length, train_label_length], y=np.zeros(len(training_img)))

# Testing the model

In [ ]:
def sig(a):
    if a[0] == "-":
        return -1
    else:
        return 1

In [ ]:
def a(l,r):
    cnt = 0
    if len(l) == len(r):
        for i in range(len(l)):
            if l[i] != r[i]:
                cnt = cnt + 1
        if cnt <= 1:
            return True
        else:
            return False
    else:
        return False

In [ ]:
from google.colab.patches import cv2_imshow
act_model.load_weights('drive/My Drive/15k toloka/0385.hdf5')
 
# predict outputs on validation images
prediction = act_model.predict(valid_img)
# print(prediction)
# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
 
# see the results
i = 0
true_num = 0
false_num = 0
for x in out:
    asd = ""
    for p in x:  
        if int(p) != -1:
            asd = asd + char_list[int(p)]
    if(asd == valid_orig_txt[i]):# or (valid_orig_txt[i][0]== "-" and  asd == valid_orig_txt[i][1:] )):
        true_num = true_num+1
        cv2_imshow((valid_img[i]*255).astype(int))
        print("pred:", asd ,"true:", valid_orig_txt[i])
    else:
        false_num = false_num+1
        # cv2_imshow((valid_img[i]*255).astype(int))
        print("pred:", asd ,"true:", valid_orig_txt[i])
    # print('\n')
    i+=1

print("Accuracy:", true_num/(true_num + false_num))

In [ ]:

# load the saved best model weights
act_model.load_weights('best_model.hdf5')
 
# predict outputs on validation images
prediction = act_model.predict(valid_img)
# print(prediction)
# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
 
# see the results
i = 0
j = 0
for x in out:
    # print("original_text =  ", valid_orig_txt[i])
    # print("predicted text = ", end = '')
    tr = ''
    for p in x:  
        if int(p) != -1:
            tr = tr + char_list[int(p)] 
            # print(char_list[int(p)], end = '')       
    # print('\n')
    print(tr, valid_orig_txt[i])
    if tr ==  valid_orig_txt[i]:
        i+=1
    else:
        j+=1
    

print(i/(i+j))

Plot of the loss function

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["loss"][20:])
plt.show()